In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import tensorflow as tf
from sklearn.metrics import classification_report
from tensorflow.keras import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.python.ops.gen_batch_ops import Batch
from tensorflow.keras.regularizers import l1_l2

2024-10-18 18:48:11.239772: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 18:48:11.261527: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 18:48:11.261547: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 18:48:11.261569: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-18 18:48:11.266095: I tensorflow/core/platform/cpu_feature_g

**Running tests on the 13 latent features extracted without applying PCA**

In [2]:
X_13 = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/13_latent_X.csv')
y_13 = pd.read_csv(r'/home/sanven0212/PycharmProjects/MultiOmicsFYP/Gleason_Scores.csv')

In [3]:
X_13.drop('Unnamed: 0', axis = 1, inplace = True)
y_13.drop('Unnamed: 0', axis = 1, inplace = True)

In [4]:
X_13

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,3.099633,2.105071,4.861546,1.201062,1.031912,2.276838,0.0,1.958306,0.0,2.132879,4.927852,3.389273,0.0
1,4.394530,1.491024,4.633244,1.322756,1.280498,0.655486,0.0,1.165946,0.0,1.597877,3.526757,2.962063,0.0
2,2.846981,1.476024,4.533834,1.592871,1.536822,2.504034,0.0,1.904280,0.0,0.728638,3.857053,2.580591,0.0
3,3.067574,1.429004,5.242934,1.838206,1.546708,2.926856,0.0,1.900046,0.0,1.008405,4.477023,3.414979,0.0
4,3.214899,1.207983,4.899215,1.441559,2.005886,2.562202,0.0,1.749290,0.0,0.353448,2.871145,2.061730,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,1.944719,0.915589,4.315665,1.593531,1.536410,3.161861,0.0,1.590570,0.0,0.064122,2.831939,1.833311,0.0
286,3.352586,1.449966,5.232137,1.512709,1.478698,2.485846,0.0,1.839456,0.0,1.321241,4.266604,3.296337,0.0
287,2.647565,1.544407,4.800531,1.838045,1.290312,2.835737,0.0,1.859061,0.0,1.010842,4.215125,2.976325,0.0
288,3.485683,1.374207,5.253157,1.617933,1.357864,2.266077,0.0,1.591797,0.0,1.400602,4.281725,3.513130,0.0


In [5]:
y_13

,Gleason_Score
0,1
1,1
2,0
3,0
4,1
...,...
285,0
286,0
287,1
288,0


In [6]:
from sklearn.model_selection import train_test_split
X_train_13, X_test_13, y_train_13, y_test_13 = train_test_split(X_13, y_13, test_size = 0.33, random_state = 42)
X_test_13, X_val_13, y_test_13, y_val_13 = train_test_split(X_test_13, y_test_13, test_size = 0.33, random_state = 42)

In [7]:
X_train_13.shape, y_train_13.shape

((194, 13), (194, 1))

In [8]:
X_test_13.shape, y_test_13.shape

((64, 13), (64, 1))

In [9]:
#ANN
from tensorflow.keras.regularizers import l2
input_layer = Input(shape = (X_train_13.shape[1], ))
x = Dense(128, activation = 'relu')(input_layer)
x = Dense(64, activation = 'relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.1)(x)
x = Dense(32, activation = 'relu')(x)
x = Dropout(0.1)(x)
output_layer = Dense(1, activation = 'sigmoid')(x)
ann_model = Model(input_layer, output_layer)
ann_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 128)               1792      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 batch_normalization (Batch  (None, 64)                256       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                             

2024-10-18 18:48:12.673311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-18 18:48:12.710366: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-18 18:48:12.710506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [10]:
ann_model.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [11]:
#Callbacks
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
lrs = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.00001, patience = 10, min_lr = 1e-6)
es = EarlyStopping(monitor = 'val_loss', patience = 11)

In [12]:
history_1 = ann_model.fit(X_train_13, y_train_13, validation_data = (X_test_13, y_test_13), epochs = 100, callbacks = [lrs, es])

Epoch 1/100


2024-10-18 18:48:13.566322: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-10-18 18:48:14.436247: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x76c830705a10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-10-18 18:48:14.436264: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-10-18 18:48:14.438938: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-18 18:48:14.446770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-10-18 18:48:14.494704: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


7/7 [==============================] - 2s 20ms/step - loss: 0.8022 - accuracy: 0.4742 - val_loss: 0.7084 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 2/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6994 - accuracy: 0.5361 - val_loss: 0.7083 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 3/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6827 - accuracy: 0.5619 - val_loss: 0.6971 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 4/100
7/7 [==============================] - 0s 4ms/step - loss: 0.6654 - accuracy: 0.6031 - val_loss: 0.6953 - val_accuracy: 0.4531 - lr: 0.0010
Epoch 5/100
7/7 [==============================] - 0s 4ms/step - loss: 0.6298 - accuracy: 0.6340 - val_loss: 0.6970 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 6/100
7/7 [==============================] - 0s 4ms/step - loss: 0.6369 - accuracy: 0.6289 - val_loss: 0.6971 - val_accuracy: 0.4688 - lr: 0.0010
Epoch 7/100
7/7 [==============================] - 0s 4ms/step - loss: 0.6348 - accuracy: 0.6340 - 

**Using recursive feature elimination with feature wrapper**

In [13]:
#Testing out the random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
rfc = RandomForestClassifier()
rfe = RFE(estimator = rfc, n_features_to_select = 5)
rfe.fit(X_train_13, y_train_13)

/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packag

RFE(estimator=RandomForestClassifier(), n_features_to_select=5)

In [14]:
print(f'Feature rank: {rfe.ranking_}')
print(f'Top features: {X_train_13.columns[rfe.support_]}')

Feature rank: [6 2 4 1 1 3 9 1 8 1 1 5 7]
Top features: Index(['3', '4', '7', '9', '10'], dtype='object')


In [15]:
X_train_rfe = rfe.transform(X_train_13)
X_val_rfe = rfe.transform(X_val_13)
X_test_rfe = rfe.transform(X_test_13)

In [16]:
X_train_rfe.shape

(194, 5)

In [17]:
rfc.fit(X_train_rfe, y_train_13)

/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [18]:
print(classification_report(y_val_13, rfc.predict(X_val_rfe)))

              precision    recall  f1-score   support

           0       0.60      0.40      0.48        15
           1       0.59      0.76      0.67        17

    accuracy                           0.59        32
   macro avg       0.60      0.58      0.57        32
weighted avg       0.60      0.59      0.58        32



In [19]:
#5 val acc-69%
#6 val acc-62%
#7 val acc-64%
#8 val acc-59%
#9 val acc-56%
#10 val acc-59%
#11 val acc-62%
#12 val acc-62%
#13 val acc-69%
#4  val acc-59%
#3  val acc-56%
#2  val acc-59%
#1  val acc-50%
#It is evident that Having 5 major features gives the same validation accuracy as having all 13 features


In [20]:
#Using only the features [2, 4, 7, 9 and 10]
X_train = X_train_13[['2', '4', '7', '9', '10']]
X_test = X_test_13[['2', '4', '7', '9', '10']]
X_val = X_val_13[['2', '4', '7', '9', '10']]

In [21]:
X_train

,2,4,7,9,10
184,6.135409,2.117820,1.037778,0.786942,2.570968
232,4.714121,1.919170,1.060741,0.250589,2.102296
38,4.632814,1.849033,2.286310,0.763380,4.068241
178,4.492625,1.444730,2.020933,0.753599,3.917394
170,4.592948,1.756269,2.191901,1.004540,4.086789
...,...,...,...,...,...
188,4.572306,1.342985,1.852130,0.694334,4.097106
71,5.278417,1.582679,1.534595,1.585902,3.956474
106,5.145191,1.242844,1.186918,1.740403,3.758870
270,4.074615,1.036103,1.995274,1.255049,4.405115


In [22]:
X_test

,2,4,7,9,10
15,5.312999,1.393442,1.789183,1.857458,4.724019
125,5.107173,0.998116,1.560883,1.827410,4.573071
180,4.455038,1.789094,2.087752,1.146413,3.831013
119,4.501893,2.008139,0.978763,0.640666,1.371113
165,4.874878,0.807901,1.396503,1.390021,4.632435
...,...,...,...,...,...
223,4.909243,1.732794,2.004662,1.277376,4.076252
10,4.582076,1.750962,1.647382,0.359396,2.785755
218,5.062508,1.842111,1.829901,0.631552,3.601160
97,5.285318,1.619579,1.544240,1.196564,3.550518


In [23]:
X_val

,2,4,7,9,10
67,5.116442,1.423162,1.366956,1.512774,3.580170
287,4.800531,1.290312,1.859061,1.010842,4.215125
68,6.079992,1.577895,0.978838,1.741999,3.666011
212,5.021482,1.693285,1.540695,0.928060,3.629627
117,5.349797,1.594493,1.625006,1.082490,3.930889
262,5.900863,1.550730,1.284157,0.600569,3.416085
93,6.746623,1.532774,0.890639,1.666480,3.982067
267,5.269382,1.356170,1.714668,1.697035,4.577945
84,3.188369,0.229596,1.189793,1.790315,3.711994
147,5.274270,1.397584,1.490384,1.475544,4.189394


In [24]:
#Testing out a neural network on these features
tf.random.set_seed(42)
#Model-1
nn_input_layer_1 = Input(shape = (X_train.shape[1], ))
x = Dense(1024, activation = 'relu')(nn_input_layer_1)
x = BatchNormalization(momentum = 0.8)(x)
x = Dense(512, activation = 'relu')(x)
x = BatchNormalization(momentum = 0.8)(x)
x = Dropout(0.1)(x)
x = Dense(256, activation = 'relu')(x)
x = BatchNormalization(momentum = 0.8)(x)
x = Dense(128, activation = 'relu')(x)
x = BatchNormalization(momentum = 0.8)(x)
nn_output_layer_1 = Dense(1, activation = 'sigmoid')(x)
nn_model_1 = Model(nn_input_layer_1, nn_output_layer_1) # This config yielded 73.71% training acc, 68.75% val acc in 23 epochs & 56% test acc
#Model-2
nn_input_layer_2 = Input(shape = (X_train.shape[1], ))
y = Dense(1024, activation = 'relu')(nn_input_layer_2)
y = BatchNormalization(momentum = 0.5)(y)
y = Dense(512, activation = 'relu')(y)
y = BatchNormalization(momentum = 0.5)(y)
y = Dense(256, activation = 'relu')(y)
y = BatchNormalization(momentum = 0.5)(y)
y = Dropout(0.1)(y)
y = Dense(128, activation = 'relu')(y)
y = Dropout(0.1)(y)
y = Dense(64, activation = 'relu')(y)
y = BatchNormalization(momentum = 0.6)(y)
y = Dense(32, activation = 'relu')(y)
y = Dropout(0.1)(y)
y = BatchNormalization(momentum = 0.7)(y)
y = Dense(16, activation = 'relu')(y)
y = BatchNormalization(momentum = 0.8)(y)
y = Dropout(0.1)(y)
y = Dense(16, activation = 'relu')(y)
y = BatchNormalization(momentum = 0.9)(y)
y = Dense(16, activation = 'relu')(y)
nn_output_layer_2 = Dense(1, activation = 'sigmoid')(y)
nn_model_2 = Model(nn_input_layer_2, nn_output_layer_2) # This config yielded 60.31% train acc, 71.88% val acc in 39 epochs

nn_input_layer_3 = Input(shape = (X_train.shape[1], ))
z = Dense(1024, activation = 'relu')(nn_input_layer_3)
z = BatchNormalization(momentum = 0.8)(z)
z = Dense(512, activation = 'relu')(z)
z = BatchNormalization(momentum = 0.8)(z)
z = Dense(256, activation = 'relu')(z)
z = BatchNormalization(momentum = 0.8)(z)
z = Dropout(0.1)(z)
z = Dense(128, activation = 'relu')(z)
z = Dropout(0.1)(z)
z = Dense(64, activation = 'relu')(z)
z = BatchNormalization(momentum = 0.8)(z)
z = Dense(32, activation = 'relu')(z)
z = Dropout(0.1)(z)
z = BatchNormalization(momentum = 0.9)(z)
z = Dense(16, activation = 'relu')(z)
z = BatchNormalization(momentum = 0.99)(z)
z = Dropout(0.1)(z)
z = Dense(16, activation = 'relu')(z)
z = BatchNormalization(momentum = 0.99)(z)
z = Dense(16, activation = 'relu')(z)
nn_output_layer_3 = Dense(1, activation = 'sigmoid')(z) 
nn_model_3 = Model(nn_input_layer_3, nn_output_layer_3) # This config gives the most stable ANN model train acc: 67.01%, val acc: 65.62% test acc: 64% 
'''nn_input_layer = Input(shape = (X_train.shape[1], ))
x = Dense(1024, kernel_regularizer = l2(0.01))(nn_input_layer)
x = BatchNormalization(momentum = 0.8)(x)
x = Activation('relu')(x)
x = Dense(512, kernel_regularizer = l2(0.01))(x)
x = BatchNormalization(momentum = 0.8)(x)
x = Activation('relu')(x)
x = Dense(256, kernel_regularizer = l2(0.01))(x)
x = BatchNormalization(momentum = 0.9)(x)
x = Activation('relu')(x)
x = Dropout(0.1)(x)
x = Dense(128, activation = 'relu', kernel_regularizer = l2(0.001))(x)
x = Dropout(0.1)(x)
x = Dense(64, kernel_regularizer = l2(0.01))(x)
x = BatchNormalization(momentum = 0.9)(x)
x = Activation('relu')(x)
x = Dense(32, kernel_regularizer = l2(0.01))(x)
x = BatchNormalization(momentum = 0.9)(x)
x = Activation('relu')(x)
x = Dropout(0.2)(x)
x = Dense(16, kernel_regularizer = l2(0.01))(x)
x = BatchNormalization(momentum = 0.99)(x)
x = Activation('relu')(x)
x = Dropout(0.3)(x)
x = Dense(16, kernel_regularizer = l2(0.01))(x)
x = BatchNormalization(momentum = 0.99)(x)
x = Activation('relu')(x)
x = Dense(16, activation = 'relu')(x)
nn_output_layer = Dense(1, activation = 'sigmoid')(x) 
nn_model = Model(nn_input_layer, nn_output_layer)''' # Using this strategy and applying it to a simple model
nn_input_layer_4 = Input(shape = (X_train.shape[1], ))
a = Dense(256)(nn_input_layer_4)
a = BatchNormalization(momentum = 0.8)(a)
a = Activation('relu')(a)
a = Dense(128)(a)
a = BatchNormalization(momentum = 0.8)(a)
a = Activation('relu')(a)
a = Dense(64)(a)
a = BatchNormalization(momentum = 0.8)(a)
a = Activation('relu')(a)
a = Dense(32)(a)
a = BatchNormalization(momentum = 0.9)(a)
a = Activation('relu')(a)
a = Dense(16)(a)
a = BatchNormalization(momentum = 0.9)(a)
a = Activation('relu')(a)
a = Dense(16)(a)
a = BatchNormalization(momentum = 0.99)(a)
a = Activation('relu')(a)
a = Dense(16)(a)
a = BatchNormalization(momentum = 0.99)(a)
a = Activation('relu')(a)
nn_output_layer_4 = Dense(1, activation = 'sigmoid')(a)
nn_model_4 = Model(nn_input_layer_4, nn_output_layer_4) # 68% and 62.5%
nn_model_4.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 5)]               0         
                                                                 
 dense_29 (Dense)            (None, 256)               1536      
                                                                 
 batch_normalization_19 (Ba  (None, 256)               1024      
 tchNormalization)                                               
                                                                 
 activation (Activation)     (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 128)               32896     
                                                                 
 batch_normalization_20 (Ba  (None, 128)               512       
 tchNormalization)                                         

In [25]:
nn_model_1.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
nn_model_2.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
nn_model_3.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
nn_model_4.compile(loss = BinaryCrossentropy(), optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [26]:
history_nn_1 = nn_model_1.fit(X_train, y_train_13, validation_data = (X_val, y_val_13), epochs = 100, callbacks = [lrs, es])

Epoch 1/100
7/7 [==============================] - 2s 20ms/step - loss: 0.9939 - accuracy: 0.5309 - val_loss: 0.6872 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 2/100
7/7 [==============================] - 0s 5ms/step - loss: 0.7644 - accuracy: 0.5876 - val_loss: 0.6374 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 3/100
7/7 [==============================] - 0s 5ms/step - loss: 0.6695 - accuracy: 0.6186 - val_loss: 0.7195 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 4/100
7/7 [==============================] - 0s 5ms/step - loss: 0.6233 - accuracy: 0.6701 - val_loss: 0.5899 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 5/100
7/7 [==============================] - 0s 4ms/step - loss: 0.6012 - accuracy: 0.6907 - val_loss: 0.4669 - val_accuracy: 0.8125 - lr: 0.0010
Epoch 6/100
7/7 [==============================] - 0s 6ms/step - loss: 0.5828 - accuracy: 0.6959 - val_loss: 0.6477 - val_accuracy: 0.5938 - lr: 0.0010
Epoch 7/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6236 - accurac

In [27]:
history_nn_2 = nn_model_2.fit(X_train, y_train_13, validation_data = (X_val, y_val_13), epochs = 100, callbacks = [lrs, es])

Epoch 1/100
7/7 [==============================] - 2s 26ms/step - loss: 0.7762 - accuracy: 0.5258 - val_loss: 0.7202 - val_accuracy: 0.4375 - lr: 0.0010
Epoch 2/100
7/7 [==============================] - 0s 8ms/step - loss: 0.7061 - accuracy: 0.5567 - val_loss: 0.5539 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 3/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6581 - accuracy: 0.5979 - val_loss: 0.6954 - val_accuracy: 0.5625 - lr: 0.0010
Epoch 4/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6537 - accuracy: 0.6340 - val_loss: 0.6805 - val_accuracy: 0.6562 - lr: 0.0010
Epoch 5/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6721 - accuracy: 0.6237 - val_loss: 0.5883 - val_accuracy: 0.6875 - lr: 0.0010
Epoch 6/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6681 - accuracy: 0.5979 - val_loss: 0.7432 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 7/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6441 - accurac

In [28]:
history_nn_3 = nn_model_3.fit(X_train, y_train_13, validation_data = (X_val, y_val_13), epochs = 100, callbacks = [lrs, es])

Epoch 1/100
7/7 [==============================] - 2s 24ms/step - loss: 0.7524 - accuracy: 0.4742 - val_loss: 0.7028 - val_accuracy: 0.4375 - lr: 0.0010
Epoch 2/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6952 - accuracy: 0.5619 - val_loss: 0.7000 - val_accuracy: 0.4375 - lr: 0.0010
Epoch 3/100
7/7 [==============================] - 0s 7ms/step - loss: 0.7184 - accuracy: 0.5619 - val_loss: 0.6636 - val_accuracy: 0.5938 - lr: 0.0010
Epoch 4/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6914 - accuracy: 0.5515 - val_loss: 0.6182 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 5/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6110 - accuracy: 0.6649 - val_loss: 0.5939 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 6/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6478 - accuracy: 0.5825 - val_loss: 0.6025 - val_accuracy: 0.7188 - lr: 0.0010
Epoch 7/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6590 - accurac

In [29]:
history_nn_4 = nn_model_4.fit(X_train, y_train_13, validation_data = (X_val, y_val_13), epochs = 100, callbacks = [lrs, es])

Epoch 1/100
7/7 [==============================] - 2s 23ms/step - loss: 0.7576 - accuracy: 0.5464 - val_loss: 0.6858 - val_accuracy: 0.4688 - lr: 0.0010
Epoch 2/100
7/7 [==============================] - 0s 5ms/step - loss: 0.6514 - accuracy: 0.6237 - val_loss: 0.6547 - val_accuracy: 0.4688 - lr: 0.0010
Epoch 3/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6745 - accuracy: 0.6082 - val_loss: 0.6432 - val_accuracy: 0.5312 - lr: 0.0010
Epoch 4/100
7/7 [==============================] - 0s 7ms/step - loss: 0.6139 - accuracy: 0.6649 - val_loss: 0.6442 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 5/100
7/7 [==============================] - 0s 6ms/step - loss: 0.6213 - accuracy: 0.6701 - val_loss: 0.6712 - val_accuracy: 0.6250 - lr: 0.0010
Epoch 6/100
7/7 [==============================] - 0s 6ms/step - loss: 0.5998 - accuracy: 0.6495 - val_loss: 0.7022 - val_accuracy: 0.6875 - lr: 0.0010
Epoch 7/100
7/7 [==============================] - 0s 8ms/step - loss: 0.6050 - accurac

In [30]:
print(classification_report(y_test_13, tf.round(nn_model_1.predict(X_test))))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.57      0.59      0.58        34
           1       0.52      0.50      0.51        30

    accuracy                           0.55        64
   macro avg       0.54      0.54      0.54        64
weighted avg       0.55      0.55      0.55        64



In [31]:
print(classification_report(y_test_13, tf.round(nn_model_2.predict(X_test))))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.55      0.79      0.65        34
           1       0.53      0.27      0.36        30

    accuracy                           0.55        64
   macro avg       0.54      0.53      0.50        64
weighted avg       0.54      0.55      0.51        64



In [32]:
print(classification_report(y_test_13, tf.round(nn_model_3.predict(X_test))))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.55      0.53      0.54        34
           1       0.48      0.50      0.49        30

    accuracy                           0.52        64
   macro avg       0.51      0.51      0.51        64
weighted avg       0.52      0.52      0.52        64



In [33]:
print(classification_report(y_test_13, tf.round(nn_model_4.predict(X_test))))

2/2 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.59      0.59      0.59        34
           1       0.53      0.53      0.53        30

    accuracy                           0.56        64
   macro avg       0.56      0.56      0.56        64
weighted avg       0.56      0.56      0.56        64



In [34]:
#XGBoost classifier
from xgboost import XGBClassifier
xgbc = XGBClassifier()
xgbc.fit(X_train, y_train_13)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [35]:
print(classification_report(y_test_13, xgbc.predict(X_test)))

              precision    recall  f1-score   support

           0       0.69      0.59      0.63        34
           1       0.60      0.70      0.65        30

    accuracy                           0.64        64
   macro avg       0.64      0.64      0.64        64
weighted avg       0.65      0.64      0.64        64



In [36]:
#Hyperparameter tuning for xgboost classifier
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}
from sklearn.model_selection import GridSearchCV
gscv = GridSearchCV(estimator = xgbc, param_grid = param_grid, scoring = 'accuracy', n_jobs = -1, cv = 5)

In [37]:
gscv.fit(X_train, y_train_13)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='accuracy')

In [38]:
print(f'Best parameters for XGBoost classifier are: {gscv.best_params_}')

Best parameters for XGBoost classifier are: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 1.0}


In [39]:
xgb = XGBClassifier(colsample_bytree = 1.0, learning_rate = 0.1, max_depth = 7, min_child_weight = 1, n_estimators = 50, subsample = 1.0)

In [40]:
xgb.fit(X_train, y_train_13)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=50, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [41]:
print(classification_report(y_test_13, xgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.67      0.59      0.62        34
           1       0.59      0.67      0.62        30

    accuracy                           0.62        64
   macro avg       0.63      0.63      0.62        64
weighted avg       0.63      0.62      0.62        64



In [42]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train_13)

/home/sanven0212/PycharmProjects/MultiOmicsFYP/.venv/lib/python3.10/site-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [43]:
print(classification_report(y_test_13, tf.round(lr.predict(X_test))))

              precision    recall  f1-score   support

           0       0.62      0.53      0.57        34
           1       0.54      0.63      0.58        30

    accuracy                           0.58        64
   macro avg       0.58      0.58      0.58        64
weighted avg       0.58      0.58      0.58        64



In [44]:
y_pred_1 = nn_model_1.predict(X_test)
y_pred_2 = nn_model_2.predict(X_test)
y_pred_3 = nn_model_3.predict(X_test)
y_pred_4 = nn_model_4.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [45]:
y_pred_1

array([[0.20772251],
       [0.83576006],
       [0.05557429],
       [0.3396494 ],
       [0.900582  ],
       [0.12441792],
       [0.85899603],
       [0.67058784],
       [0.35401544],
       [0.0625273 ],
       [0.4136603 ],
       [0.04845815],
       [0.67261696],
       [0.33412188],
       [0.4280171 ],
       [0.40360117],
       [0.63350505],
       [0.84803486],
       [0.13325922],
       [0.07094293],
       [0.98036283],
       [0.9159753 ],
       [0.38302964],
       [0.7340333 ],
       [0.55171674],
       [0.07832527],
       [0.5944525 ],
       [0.09057747],
       [0.5465478 ],
       [0.44178587],
       [0.6728743 ],
       [0.18448569],
       [0.3513235 ],
       [0.19097854],
       [0.66251516],
       [0.532544  ],
       [0.07764928],
       [0.86756337],
       [0.46341333],
       [0.43172818],
       [0.529853  ],
       [0.8690169 ],
       [0.3661182 ],
       [0.86972713],
       [0.27045777],
       [0.39426026],
       [0.8586984 ],
       [0.694

In [46]:
y_pred_2

array([[0.09942433],
       [0.2448363 ],
       [0.09497927],
       [0.43401042],
       [0.5568208 ],
       [0.19981022],
       [0.9182832 ],
       [0.22948727],
       [0.3555187 ],
       [0.28337336],
       [0.52299255],
       [0.1301128 ],
       [0.51256526],
       [0.22430398],
       [0.29402155],
       [0.21782663],
       [0.3352142 ],
       [0.5887369 ],
       [0.27098796],
       [0.43025044],
       [0.85225767],
       [0.69482046],
       [0.1311086 ],
       [0.759667  ],
       [0.19738956],
       [0.12222442],
       [0.47796735],
       [0.29470867],
       [0.3962247 ],
       [0.44868028],
       [0.29496524],
       [0.37629306],
       [0.2535225 ],
       [0.32052162],
       [0.75364465],
       [0.23080419],
       [0.03251144],
       [0.5912936 ],
       [0.24807933],
       [0.50657195],
       [0.30848828],
       [0.71835315],
       [0.30983996],
       [0.39642325],
       [0.21315011],
       [0.31299907],
       [0.20935258],
       [0.215

In [48]:
y_test_13

,Gleason_Score
15,0
125,0
180,0
119,1
165,1
...,...
223,1
10,1
218,0
97,1


In [49]:
y_pred_3

array([[0.1907587 ],
       [0.4700643 ],
       [0.28183836],
       [0.59741616],
       [0.5322646 ],
       [0.14446282],
       [0.6059151 ],
       [0.4998296 ],
       [0.48789766],
       [0.37133726],
       [0.5698519 ],
       [0.24050085],
       [0.59246546],
       [0.48979264],
       [0.47603983],
       [0.4613793 ],
       [0.4365323 ],
       [0.63728464],
       [0.29673654],
       [0.3781914 ],
       [0.7020716 ],
       [0.6254778 ],
       [0.52522373],
       [0.55306435],
       [0.4972265 ],
       [0.4889639 ],
       [0.5287632 ],
       [0.50440824],
       [0.34031576],
       [0.47922918],
       [0.2437665 ],
       [0.60165787],
       [0.43771136],
       [0.60120964],
       [0.58834237],
       [0.14426865],
       [0.30833986],
       [0.6518088 ],
       [0.37613097],
       [0.5150886 ],
       [0.7332451 ],
       [0.6321622 ],
       [0.60811603],
       [0.6183773 ],
       [0.38716367],
       [0.425567  ],
       [0.48692888],
       [0.608

In [59]:
y_pred_4

array([[0.3825493 ],
       [0.59731287],
       [0.36999017],
       [0.35127306],
       [0.48651183],
       [0.00370971],
       [0.86513066],
       [0.7272983 ],
       [0.26794663],
       [0.1426257 ],
       [0.18224809],
       [0.2896781 ],
       [0.64730847],
       [0.7898674 ],
       [0.16090961],
       [0.57267135],
       [0.3167144 ],
       [0.71885526],
       [0.2601204 ],
       [0.2371213 ],
       [0.92504627],
       [0.44899127],
       [0.2248646 ],
       [0.73560894],
       [0.56744057],
       [0.5095598 ],
       [0.19833043],
       [0.2819088 ],
       [0.26031336],
       [0.7779786 ],
       [0.15733916],
       [0.34041697],
       [0.31473005],
       [0.20070341],
       [0.56804436],
       [0.09390738],
       [0.13719432],
       [0.8580171 ],
       [0.6292913 ],
       [0.7244398 ],
       [0.7659817 ],
       [0.5634239 ],
       [0.6861398 ],
       [0.79517907],
       [0.24512005],
       [0.33011627],
       [0.48709202],
       [0.622

In [62]:
results_dict = {'model_1': tf.squeeze(y_pred_1), 'model_2': tf.squeeze(y_pred_2), 'model_3': tf.squeeze(y_pred_3), 'model_4': tf.squeeze(y_pred_4)}

In [63]:
results_df = pd.DataFrame(results_dict)

In [64]:
results_df

,model_1,model_2,model_3,model_4
0,0.207723,0.099424,0.190759,0.382549
1,0.835760,0.244836,0.470064,0.597313
2,0.055574,0.094979,0.281838,0.369990
3,0.339649,0.434010,0.597416,0.351273
4,0.900582,0.556821,0.532265,0.486512
...,...,...,...,...
59,0.058957,0.036852,0.163642,0.253214
60,0.892860,0.435861,0.620107,0.684802
61,0.537017,0.304281,0.486960,0.590154
62,0.646067,0.555288,0.561777,0.416307


In [65]:
results_df['model_1'] = results_df['model_1'].apply(lambda x: 1 if x > 0.5 else 0)
results_df['model_2'] = results_df['model_2'].apply(lambda x: 1 if x > 0.5 else 0)
results_df['model_3'] = results_df['model_3'].apply(lambda x: 1 if x > 0.5 else 0)
results_df['model_4'] = results_df['model_4'].apply(lambda x: 1 if x > 0.5 else 0)

In [66]:
results_df

,model_1,model_2,model_3,model_4
0,0,0,0,0
1,1,0,0,1
2,0,0,0,0
3,0,0,1,0
4,1,1,1,0
...,...,...,...,...
59,0,0,0,0
60,1,0,1,1
61,1,0,0,1
62,1,1,1,0


In [67]:
results_df['pred_label'] = results_df.apply(lambda x: x.mode()[0], axis = 1)

In [68]:
results_df

,model_1,model_2,model_3,model_4,pred_label
0,0,0,0,0,0
1,1,0,0,1,0
2,0,0,0,0,0
3,0,0,1,0,0
4,1,1,1,0,1
...,...,...,...,...,...
59,0,0,0,0,0
60,1,0,1,1,1
61,1,0,0,1,0
62,1,1,1,0,1


In [72]:
results_df['True_label'] = tf.squeeze(y_test_13)

In [73]:
results_df

,model_1,model_2,model_3,model_4,pred_label,True_label
0,0,0,0,0,0,0
1,1,0,0,1,0,0
2,0,0,0,0,0,0
3,0,0,1,0,0,1
4,1,1,1,0,1,1
...,...,...,...,...,...,...
59,0,0,0,0,0,1
60,1,0,1,1,1,1
61,1,0,0,1,0,0
62,1,1,1,0,1,1


In [74]:
print(classification_report(results_df['True_label'], results_df['pred_label']))

              precision    recall  f1-score   support

           0       0.59      0.76      0.67        34
           1       0.60      0.40      0.48        30

    accuracy                           0.59        64
   macro avg       0.60      0.58      0.57        64
weighted avg       0.60      0.59      0.58        64



In [ ]:
#Training a classifier on this dataframe
